In [1]:
import torch
import torch.distributions as dist
import torch.nn as nn
import torch.nn.functional as Fcn
import torch.optim as optim

In [2]:
def decimal_to_binary_tensor(value, width=0):
    string = format(value, '0{}b'.format(width))
    binary = [0 if c == '0' else 1 for c in string]
    return torch.tensor(binary, dtype=torch.float)

In [70]:
# Creating Ising spins and Calculating Hamiltonian of the Ising system
class Ising_model():
    
    def __init__(self, num_rows, num_cols):
        self.num_rows, self.num_cols = num_rows, num_cols
        self.num_spins = num_rows * num_cols
        self.beta = 1.
        self.J = self.magnetization()
    
    def neighbors(self, i, j):
        nhb = []
        if i > 0:
            nhb.append([i-1, j])
        if i < self.num_rows - 1:
            nhb.append([i+1, j])
        if j > 0:
            nhb.append([i, j-1])
        if j < self.num_cols - 1:
            nhb.append([i, j+1])
        return nhb
                    
    def magnetization(self):
        J = torch.zeros(self.num_spins, self.num_spins)
        for i in range(self.num_rows):
            for j in range(self.num_cols):
                for i_nhb, j_nhb in self.neighbors(i, j):
                    J[self.num_rows * i + j, self.num_rows * i_nhb + j_nhb] = 1                
        return J
    
    def hamiltonian(self, spin_state):
        s = spin_state
        J = self.J
        H = - torch.matmul(s, torch.matmul(J, s)) / 2
        return H
    
    def Energy_expectation(self):
        num_state = 2**(self.num_spins)
        Z = 0.
        E_true = 0.
        for i_state in range(num_state):
            binary = decimal_to_binary_tensor(i_state, width=self.num_spins)
            spin_state = 1 - 2 * binary
            H = self.hamiltonian(spin_state)
            Z += torch.exp(-self.beta * H)
        for i_state in range(num_state):
            binary = decimal_to_binary_tensor(i_state, width=self.num_spins)
            spin_state = 1 - 2 * binary
            H = self.hamiltonian(spin_state)
            E_true += H * torch.exp(- self.beta * H) / Z
        return E_true

In [71]:
class Ising_sample():
    
    def __init__(self, data_size, num_rows, num_cols):
        self.ising = Ising_model(num_rows, num_cols)
        self.data_size = data_size
        self.data_length = num_rows * num_cols
        self.num_rows, self.num_cols = num_rows, num_cols
        self.dataset = self.Metropolis_Sampling()
    
    def gen_sample(self, p = 0.5):
        probs = torch.rand(self.num_rows, self.num_cols)
        sample = torch.where(probs < p, torch.zeros(1), torch.ones(1))
        return sample
    
    # create a training data set by using Metropolis algorithm
    def Metropolis_Sampling(self):
        beta = self.ising.beta
        for i in range(self.data_size):
            # randomly sample a ising model
            sample_2d = self.gen_sample()
            # reshape a 2d sample to a 1d tensor as the visible layer
            v = sample_2d.view(-1)
            # transform the sample to ising spins:  0 ---> spin +1,    1 --> spin -1
            spin_state = 1 - 2 * v
            # calculate the Hamiltonian of the model
            H_next = self.ising.hamiltonian(spin_state)
        
            # Monte Carlo sampling by Metropolis algorithm
            if i == 0:
                dataset = v.unsqueeze(0)
            else:
                delta_H = H_next - H_current
                if delta_H <= 0:
                    dataset = torch.cat((dataset, v.unsqueeze(0)), dim = 0)
                else:
                    ber = torch.bernoulli(torch.exp(- beta * delta_H))
                    if ber == 1:
                        dataset = torch.cat((dataset, v.unsqueeze(0)), dim = 0)
                    else:
                        dataset = torch.cat((dataset, dataset[i-1].unsqueeze(0)), dim = 0)
            H_current = H_next
        return dataset
    
    def prob_data(self):
        num_state = 2**self.data_length
        count = torch.zeros(num_state)
        for i_state in range(num_state):
            for i_data in range(self.data_size):
                bin_state = decimal_to_binary_tensor(i_state, width=self.data_length)
                if torch.all(torch.eq(self.dataset[i_data], bin_state)) == 1:
                    count[i_state] += 1
        prob = count / self.data_size
        return prob
    
    def Entropy_data(self):
        num_state = 2**self.data_length
        prob = self.prob_data()
        Entropy = 0.
        for i_state in range(num_state):
            if prob[i_state] > 0:
                Entropy -= prob[i_state] * torch.log(prob[i_state])  
        return Entropy
    
    def Energy_data(self):
        spinset = 1 - 2 * self.dataset
        E = torch.zeros(self.data_size)
        for i_data in range(self.data_size):
            E[i_data] = self.ising.hamiltonian(spinset[i_data]).item()
        E_data = torch.mean(E)
        return E_data

In [72]:
# Creating the RBM Architecture (weights, biases)
class RBM():
    
    # Initiate RBM parameters
    def __init__(self, nv, nh):
        self.nv = nv
        self.nh = nh
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(nh)
        self.b = torch.randn(nv)
    
    def Energy(self, v, h): 
        ah = torch.matmul(self.a, h)
        bv = torch.matmul(self.b, v)
        hWv = torch.matmul(h, torch.matmul(self.W, v))
        E = - ah - bv - hWv
        return E
    
    def FreeEnergy(self, v):
        bv = torch.matmul(self.b, v)
        Wv = torch.matmul(self.W, v)
        sp = nn.Softplus()
        F = - bv - torch.sum(sp(self.a + Wv))
        return F
    
    def Partition_function(self):
        num_state = 2**self.nv
        Z = 0.
        for idx_state in range(num_state):
            v = decimal_to_binary_tensor(idx_state, width=self.nv)
            Z += torch.exp(-self.FreeEnergy(v))
        return Z
    
    # Calculate Negative Log-Likelihood
    def NLL(self, Data):
        data_size = Data.size()[0]
        Z = self.Partition_function()        
        F = torch.zeros(data_size)
        for i in range(data_size):
            F[i] = self.FreeEnergy(Data[i]).item()
        NLL = torch.mean(F) + torch.log(Z)
        return NLL
    
    # Contrastive Divergence
    def update_CD(self, Batch, learning_rate):
        batch_size = Batch.size()[0]
        delta_W = torch.zeros_like(self.W)
        delta_a = torch.zeros_like(self.a)
        delta_b = torch.zeros_like(self.b)
        for i in range(batch_size):
            a = self.a
            b = self.b
            num_iterations = 10
            
            # k-iterations of Gibbs sampling
            for k in range(num_iterations):
                if k == 0:
                    v_0 = Batch[i]
                    Wv = torch.matmul(self.W, v_0)
                    p_h_given_v_0 = torch.sigmoid(a + Wv)
                    h_0 = torch.bernoulli(p_h_given_v_0)
                    h_old = h_0
                else:
                    h_old = h_new
                
                hW = torch.matmul(torch.t(self.W), h_old)
                p_v_given_h = torch.sigmoid(b + hW)
                v_new = torch.bernoulli(p_v_given_h)
                Wv = torch.matmul(self.W, v_new)
                p_h_given_v = torch.sigmoid(a + Wv)
                h_new = torch.bernoulli(p_h_given_v)
            
            # update parameters after k-iterations of Gibbs sampling
            delta_W += learning_rate * (torch.ger(p_h_given_v_0, v_0) - torch.ger(p_h_given_v, v_new))
            delta_a += learning_rate * (p_h_given_v_0 - p_h_given_v)
            delta_b += learning_rate * (v_0 - v_new)
            
        self.W += delta_W / batch_size
        self.a += delta_a / batch_size
        self.b += delta_b / batch_size

In [73]:
# set data size
num_rows = 2
num_cols = 2
data_size = 1000
num_spins = num_rows * num_cols

# set RBM parameters
nv = num_spins
nh = 10
rbm = RBM(nv, nh)

In [75]:
sampling = Ising_sample(data_size, num_rows, num_cols)
D = sampling.dataset
print('D = {}'.format(D))
E = sampling.Energy_data()
print('E = {}'.format(E))
model = Ising_model(num_rows, num_cols)
H = model.Energy_expectation()
print('H = {}'.format(H))

D = tensor([[0., 1., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [1., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 1., 1.]])
E = -0.972000002861023
H = -3.6016509532928467


In [67]:
# Train the RBM
num_epoch = 100
#batch_size = 100
learning_rate = 1e-3

for epoch in range(0, num_epoch + 1):
    sampling = Ising_sample(data_size, num_rows, num_cols)
    D = sampling.dataset
    S = sampling.Entropy_data()
    if epoch > 0:
#        for batch_idx in range(int(data_size / batch_size)):
#            Batch = D[batch_idx * batch_size : (batch_idx + 1) * batch_size]
        rbm.update_CD(D, learning_rate)
    loss = rbm.NLL(D) - S
    print('epoch {}: loss = {}'.format(epoch, loss))

epoch 0: loss = 3.2910714149475098
epoch 1: loss = 3.2321252822875977
epoch 2: loss = 3.072360038757324
epoch 3: loss = 3.228456974029541
epoch 4: loss = 3.2955567836761475
epoch 5: loss = 3.2968311309814453
epoch 6: loss = 3.153411388397217
epoch 7: loss = 3.069809913635254
epoch 8: loss = 3.3044538497924805
epoch 9: loss = 3.3686604499816895
epoch 10: loss = 3.163126230239868
epoch 11: loss = 3.097424268722534
epoch 12: loss = 3.195131540298462
epoch 13: loss = 2.9905457496643066
epoch 14: loss = 3.3045527935028076
epoch 15: loss = 3.0767908096313477
epoch 16: loss = 3.1711554527282715
epoch 17: loss = 3.1516358852386475
epoch 18: loss = 3.196458578109741
epoch 19: loss = 3.108738899230957
epoch 20: loss = 3.258394718170166
epoch 21: loss = 3.18953800201416
epoch 22: loss = 3.1629414558410645
epoch 23: loss = 3.0662074089050293
epoch 24: loss = 3.126600980758667
epoch 25: loss = 3.0537891387939453
epoch 26: loss = 3.211574077606201
epoch 27: loss = 2.9959802627563477
epoch 28: loss =

In [198]:
# sampling after training
for k in range(10000):
    if k == 0:
        v_0 = torch.bernoulli(torch.rand(nv))
        Wv = torch.matmul(rbm.W, v_0)
        p_h_given_v_0 = torch.sigmoid(rbm.a + Wv)
        h_0 = torch.bernoulli(p_h_given_v_0)
        h_old = h_0
    else:
        h_old = h_new
                
    hW = torch.matmul(torch.t(rbm.W), h_old)
    p_v_given_h = torch.sigmoid(rbm.b + hW)
    v_new = torch.bernoulli(p_v_given_h)
    Wv = torch.matmul(rbm.W, v_new)
    p_h_given_v = torch.sigmoid(rbm.a + Wv)
    h_new = torch.bernoulli(p_h_given_v)
    
    if k%1000==999:
        print(v_new)

tensor([0., 0., 1., 1.])
tensor([0., 0., 1., 0.])
tensor([0., 1., 1., 0.])
tensor([1., 0., 0., 1.])
tensor([0., 1., 1., 0.])
tensor([1., 1., 1., 1.])
tensor([0., 0., 1., 1.])
tensor([1., 0., 0., 1.])
tensor([0., 0., 1., 0.])
tensor([1., 1., 0., 1.])
